# Python-MATLAB interaction

## Requirements

In order to call MATLAB functions from Python you need
  1. a Python distribution, e.g., [Anaconda](https://www.anaconda.com/distribution/), and
  1. [MATLAB](https://nl.mathworks.com/); here we are using R2017b.
  
As usual, it is prudent to create a new conda environment, since this requires some Python packages to be installed.  Once done, use iPython to navigate to the directory in which MATLAB is installed.  Once there, change directory to `extern/engines/python`.

Execute the easy_install script in this directory, i.e.,
```bash
$ python setup.py install
```

## Starting and stopping the MATLAB engine

The interaction with MATLAB is accomplished through the MATLAB engine. The engine has to be started before doing any calls to MATLAB functions, and can be stopped when done.

The appropriate module has to be loaded first.

In [1]:
import matlab.engine

Next, the engine can be started. Note that this can take a while...

In [2]:
engine = matlab.engine.start_matlab()

Now MATLAB functions can be called, e.g., `isprime` that returns `True` if its argument is a prime number, false otherwise.

In [3]:
engine.isprime(37)

True

In [4]:
engine.isprime(36)

False

Once all MATLAB function calls are done, the engine can be stopped.

In [5]:
engine.quit()

## Vectors and matrices

Import the MATLAB engine module and start the engine.

In [6]:
import matlab.engine
engine = matlab.engine.start_matlab()

Create two MATLAB vectors from within Python.

In [4]:
a = matlab.double([1.0, 3.0, -2.0])
b = matlab.double([2.0, -1.0, 1.0])

Compute the scalar product of the two vectors.

In [5]:
engine.dot(a, b)

-3.0

Create a matrix.

In [6]:
A = matlab.double([[1.0, 3.0], [4.0, -2.0]])

Compute the singular value decomposition of the matrix `A`.

In [7]:
U, S, V = engine.svd(A, nargout=3)

Note the `nargout` optional argument, if this is left out, the `svd` function will only return the singular values.

Reconstruct the original matrix from `U`, `S`, and `V`.

In [33]:
A_r = engine.mtimes(U, engine.mtimes(S, engine.transpose(V)))

In [34]:
for A_row in A_r:
    print(A_row)

[1.0000000000000004,3.0000000000000013]
[4.000000000000001,-2.0000000000000004]


Alternatively, we can store `U`, `S`, and `V` in MATLAB's workspace, and evaluate an expression.

In [8]:
engine.workspace['U'] = U
engine.workspace['S'] = S
engine.workspace['V'] = V

In [9]:
engine.eval("U*S*V.'")

matlab.double([[1.0000000000000004,3.000000000000001],[4.0,-2.0]])

## Executing MATLAB scripts and functions from Python

Import the MATLAB engine module and start the engine.

In [1]:
import matlab.engine
engine = matlab.engine.start_matlab()

We want to compute the value of `A*x + b`, where `A` is a 2 by 2 matrix, and `x` and `b` are column vectors of size 2. First define `A`, `x`, and `b`, and store them in MATLAB's workspace.

In [20]:
A = matlab.double([[3.0, -1.0], [2.0, 5.0]])
x = matlab.double([[1.0], [2.0]])
b = matlab.double([[-7.0], [2.0]])
engine.workspace['A'] = A
engine.workspace['x'] = x
engine.workspace['b'] = b

The script `linear_script.m` computes `y = A*x + b`. This will store the result in `y`, which is a variable in MATLAB's workspace, from where we can retrieve it.

In [21]:
engine.linear_script(nargout=0)

In [22]:
print(engine.workspace['y'])

[[-6.0],[14.0]]


To verify, simply evaluate the expression `A*x + y`, indeed, the result is identival.

In [23]:
engine.eval('A*x + b')

matlab.double([[-6.0],[14.0]])

The file `linear_function.m` defines a single MATLAB function `linear_function(A, x, b) = A*x + b`. We can call this with the MATLAB matrices we defined previously to obtain again the same result.

In [24]:
engine.linear_function(A, x, b)

matlab.double([[-6.0],[14.0]])